In [1]:
import xarray as xr
import os 
import pandas as pd

In [2]:
# function for rounding to nearest 1/4
def x_rnd(x):
    return round(x*4)/4

In [3]:
# path to annual wheat yields data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/gro_ylds/wheatf_mn_ia.xlsx'
y = pd.read_excel(path)

In [4]:
# path to monthly relative humidity data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/era5/monthly/'

# retrieve all files in dataset
files = []
for file in os.listdir(path):
    if file.endswith('nc'):
        files.append(os.path.join(path, file))

# open files as xarray
ds = xr.open_mfdataset(files)

In [18]:
# store monthly mean uv at t/lat/lon where yield data is recorded
cols = ['yr', 'lat', 'lon', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
rh = pd.DataFrame(index = range(0, len(y.index)), columns=cols)

for ix in y.index:
    
    lat = y.loc[ix].lat     # retrieve latitude at this entry
    lat = x_rnd(lat)       # round lat coord to nearest 0.25
    
    lon = y.loc[ix].lon     # retrieve longitude at this entry
    lon = x_rnd(lon)        # round lon coord to nearest 0.25
    
    yr = y.loc[ix].StartDate[0] + y.loc[ix].StartDate[1] + y.loc[ix].StartDate[2] + y.loc[ix].StartDate[3]
    
    # rh2m at this lat/lon
    rh2m = ds.RH2m.sel(latitude=lat, longitude=lon)
    
    # store monthly mean uv
    for i in range(1, 13):
        t = yr + '-' + str(i)
        rh.iat[ix, i+2] = rh2m.sel(time=t).compute().values[0]
    
    # store yr for this point
    col = 0
    rh.iat[ix, col] = yr
    
    # store latitude for this point
    col = 1
    rh.iat[ix, col] = lat
    
    # store longitude for this point
    col = 2
    rh.iat[ix, col] = lon

#spath = '/Users/demetrayancopoulos/Desktop/SML310/final_project/monthly_rh_mean.xlsx'
#rh.to_excel(spath)
